In [1]:
from ginkgo.data import *
import numpy as np

In [2]:
stock_list = get_stockinfos()

没有缓存，重新计算并缓存


🗿 Got 5375 records about StockInfo from mysql.

In [3]:
stock_code = "000002.SZ"
stock_info = stock_list[stock_list["code"]==stock_code]
display(stock_info)

,code,code_name,industry,currency,market,list_date,delist_date,uuid,meta,desc,create_at,update_at,is_del,source
268,000002.SZ,万科A,全国地产,CURRENCY_TYPES.CNY,MARKET_TYPES.CHINA,1991-01-29,2100-01-01,0dd3cddd92c246698ed3f6aa5b1521f0,{},"This man is lazy, there is no description.",2024-12-04 17:33:10,2024-12-09 00:17:25,False,SOURCE_TYPES.TUSHARE


In [4]:
date_range = 5

In [5]:
start_date = "2000-01-01"
end_date = "2025-01-01"

In [6]:
bar_data_raw = get_bars(code=stock_code, start_date=start_date, end_date=end_date, as_dataframe=True)
display(bar_data_raw.head(2))
bar_data_raw.describe()

,code,open,high,low,close,volume,amount,frequency,uuid,meta,desc,timestamp,source
0,000002.SZ,9.80,10.40,9.6,10.30,45747,46053.45,FREQUENCY_TYPES.DAY,4413c7238c5a44d08061fea0852f2369,{}                                            ...,"This man is lazy, there is no description.    ...",2000-01-04,SOURCE_TYPES.DATABASE
1,000002.SZ,10.35,10.45,10.0,10.04,46137,47348.00,FREQUENCY_TYPES.DAY,a4ecc6096037411281e04e2a5db1dafa,{}                                            ...,"This man is lazy, there is no description.    ...",2000-01-05,SOURCE_TYPES.DATABASE


,open,high,low,close,volume,amount,timestamp
count,5845.000000,5845.000000,5845.000000,5845.000000,5.845000e+03,5.845000e+03,5845
mean,14.630479,14.906712,14.389321,14.647902,7.818154e+05,1.154262e+06,2012-07-08 08:42:32.335329536
min,3.140000,3.200000,3.120000,3.140000,2.334000e+03,2.566000e+03,2000-01-04 00:00:00
25%,8.250000,8.380000,8.140000,8.270000,2.467280e+05,2.164410e+05,2006-04-11 00:00:00
50%,12.300000,12.520000,12.100000,12.300000,5.767120e+05,7.976183e+05,2012-05-15 00:00:00
75%,20.430000,20.730000,20.110000,20.500000,9.688210e+05,1.593419e+06,2018-12-21 00:00:00
max,41.120000,42.240000,40.460000,41.130000,1.097234e+07,2.010649e+07,2024-12-31 00:00:00
std,8.082726,8.247040,7.942681,8.093063,8.785911e+05,1.327240e+06,NaN


In [7]:
feature_columns = ['open', 'high', 'low', 'close', 'volume', 'amount']

df = bar_data_raw[feature_columns].copy()

In [8]:
df.loc[:,f'hh_{date_range}'] = df['high'].rolling(window=date_range).max()

In [9]:
df.loc[:,f'lc_{date_range}'] = df['close'].rolling(window=date_range).min()

In [10]:
df.loc[:,f'hc_{date_range}'] = df['close'].rolling(window=date_range).max()

In [11]:
df.loc[:,f'll_{date_range}'] = df['low'].rolling(window=date_range).max()

In [14]:
df = df.dropna().reset_index(drop=True).copy()
df['range'] = df.apply(lambda row: max(row[f'hh_{date_range}'] - row[f'lc_{date_range}'],
                                       row[f'hc_{date_range}'] - row[f'll_{date_range}']), axis=1)


In [16]:
k1 = 0.5 # 0.1~1
k2 = 0.5 # 0.1~1

In [17]:
df['buy_line'] = df['open'] + k1 * df['range']

In [18]:
df['sell_line'] = df['open'] + k2 * df['range']

In [19]:
df

,open,high,low,close,volume,amount,hh_5,lc_5,hc_5,ll_5,range,buy_line,sell_line
0,10.68,11.44,10.55,11.44,142425,159259.05,11.44,10.04,11.44,10.55,1.40,11.380,11.380
1,11.60,11.70,10.65,10.84,111362,125253.89,11.70,10.04,11.44,10.65,1.66,12.430,12.430
2,10.79,11.12,10.39,10.46,46986,50058.55,11.70,10.46,11.44,10.65,1.24,11.410,11.410
3,10.39,10.70,10.35,10.44,27679,29140.94,11.70,10.44,11.44,10.65,1.26,11.020,11.020
4,10.60,10.60,10.00,10.03,18015,18242.96,11.70,10.03,11.44,10.65,1.67,11.435,11.435
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5836,7.71,7.73,7.47,7.54,1517662,1146549.84,8.01,7.54,7.91,7.82,0.47,7.945,7.945
5837,7.50,7.56,7.48,7.51,710188,534064.46,7.93,7.51,7.82,7.80,0.42,7.710,7.710
5838,7.51,7.63,7.49,7.55,1004323,760189.35,7.80,7.51,7.73,7.65,0.29,7.655,7.655
5839,7.50,7.51,7.33,7.36,1338890,990277.59,7.73,7.36,7.73,7.63,0.37,7.685,7.685


In [20]:
# 如果上穿 buy_line, 开多
# 如果下穿 sell_line, 开空